# EEIO Exercise
<a target="_blank" href="https://colab.research.google.com/github/jwardbond/spm_io_meeting/blob/master/spm_io_exercise_distribute.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook walks through some basic calculations with the EXIOBASE Input-Output model. Working through it, you will learn:
- The structure of EXIOBASE tables.
- How to calculate the Leontief inverse $L$ using the demand model.
- Intuition on how to diagonalize and isolate regions in an MRIO model.
- How to calculate the consumption and production impacts of Canada.
- How to calculate scope 1, 2, and 3 emissions for a region.
<br><br>
This tutorial requires numpy, pandas, and matplotlib packages. If you do not have these installed, please run `!pip install <package name>` in your python env.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Input data
To complete this exercise, you will need the 2019 product by product tables from Exiobase. All Exiobase time series can be downloaded at the following repository: https://zenodo.org/doi/10.5281/zenodo.3583070
<br><br>
Download and extract the `IOT_2019_pxp.zip` file. Alternatively (if the file sizes are too large), you can just save `A.txt`, `y.txt`, `F.txt`, `F_y.txt`
<br><br>
Put the files in your current directory. The code below imports the tables we will use for calculations. Note that these are multi-index and multi-column pandas dataframes; indexing with Exiobase can be tricky and its best to drop to a numpy matrix when performing calculations.

In [ ]:
!wget https://zenodo.org/records/5589597/files/IOT_2019_pxp.zip?download=1
!unzip  /content/IOT_2019_pxp.zip

In [ ]:
# Import data from Exiobase 2019 product by product
path = '/content/IOT_2019_pxp/' 

A = pd.read_csv(f'{path}A.txt', sep='\t', index_col=[0, 1], header=[0, 1])
Y = pd.read_csv(f'{path}Y.txt', sep='\t', index_col=[0, 1], header=[0, 1])
F = pd.read_csv(f'{path}impacts/F.txt', sep='\t', index_col=[0], header=[0, 1])
F_y = pd.read_csv(f'{path}impacts/F_Y.txt', sep='\t', index_col=[0], header=[0, 1]) 

Lets look at these tables. Another good thing to do when performing EEIO analysis is to keep track of your vector and matrix dimensions. These will help you verify your matrix operations later on.

In [ ]:
# dimensions
print('A:', A.shape,'Y:', Y.shape,'F:', F.shape,'Fy:', F_y.shape)
# Uncomment a table you would like to look at...
# the total industry by industry use matrix
A
# the final demand of the economy
#Y
# the impacts of industry by industry activity
#F
# the final demand impacts
#F_y

## 2. Demand model calculations
First, we will construct our missing tables using Leontief's demand model. We need the inverse $L$ and the total economic output $x$. Recall that $L = (I-A)^{-1}$ and $x=LY$. We will use some numpy operators (@ is pythons matmul operator).
<br><br>
Note that we sum all of the columns of final demand $Y$. You could leave these disaggregated and see the final output of each final demand sector, but usually we want x to be a single vector for each product and country, so we reduce these dimensions.
<br><br>
check: if everything has gone correct, the total output of the world's economy in Exiobase should be 157028217.75839323 (millions of Euros), or ~235 Trillion Canadian dollars.

In [ ]:
# we create an identity matrix of the same shape as A
# this might take a while to run.
I = np.identity(A.shape[0])
L = np.linalg.inv(I-A)

In [ ]:
x = L @ Y.sum(axis=1)

# Check the dimensions
print('matmul shape: L', L.shape, 'Y_sum', Y.sum(axis=1).shape)
print('x shape', x.shape)
x.sum()

## 3. EEIO production and consumption emissions
Now that we have all of the variables in the demand model, we can use the environmental extension tables $F$ and $F_y$ to calculate the GWP100 of Canada (and any other region or sector combination in the world).
<br><br>
The standard formulas are the same for any impact calculation: <br>
Production impacts $\rightarrow$ $p = F + F_y$  <br>
Consumption impacts $\rightarrow$ $q=fLY + F_{y}$ where the dimensions of the calculation are as follows 
<br>
$\underset{j \times mn}{f} \times \underset{mn \times mn}{L} \times \underset{mn \times n}{Y} + \underset{j \times n}{F_{hh}}$ 
<br>

The level of detail will vary depending on how much you aggregate the tables in the matrix calculation.

In [ ]:
x_prime = x.copy()

First, we need $f = Fx^{-1}$ (normalize impacts by total output). We need to diagonalize x for this. Because x diag maybe have zeros along its diagonal, we can't use the basic numpy operators to do this (singular inversion). Instead we just find $1/x$ for the non-zero values; this is functionally the same as inverting the diagonalized vector.

In [ ]:
# get 1/x
x_prime[x_prime != 0] = 1/x_prime[x_prime != 0] 

# diagonalize x
inv_diag_x = np.diag(x_prime)

# We can now get the outputs per monetary unit of output (kg/euro)
f = F @ inv_diag_x
print(f.shape)

In [ ]:
# We make a copy of our product output vector to prevent changing x!
x_ = x.copy() 

# we divide 1 by the values that are non-0
x_[x_!=0] = 1/x_[x_!=0]

# We diagolize the resulting vector
inv_diag_x = np.diag(x_)

# We are essentially dividing the total stressors or impacts (F) by the product output (e.g., kg/euro)
f = F @ inv_diag_x

Lets do the footprint calculation. We need to find the row in f (out of 126 impacts) that we are interested in -> this will be the IPCC GHG emissions 100GWP. We also have to clean and aggregate some of the variables: Since we are just looking for Canada's total impacts, we can group by country level and ignore all sectors. <br>
From consumption formula above, our final value $q$ will be a $j \times n$ dimension table, where $j$ is the impact and $n$ is the sectoral/country aggregation. So, we want to modify the $n$ dimension of $Y$ and $F_y$ to get our desired level (We also need an aggregated $F$ matrix for the production calculation)

In [ ]:
# Get the f column we are interested in
impact = 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'

# Aggregate Y and F_Y. We tranpose and then transpose back to use pandas groupby operator
Y_agg = Y.T.groupby(['region'], sort=False).sum().T
F_agg = F.T.groupby(["region"], sort=False).sum().T
F_y_agg = F_y.T.groupby(["region"], sort=False).sum().T

# you can also do it this way using multindexing:
#Y_agg2 = Y.groupby(axis=1, level=0).sum()

# There are 49 regions in Exiobase
print("aggregated Y", Y_agg.shape, "aggregated F_y", F_y_agg.shape, "aggregated F", F_agg.shape)
Y_agg.head()

Finally, lets do the calculation.

In [ ]:
# Production impacts or territorial accounts (PBA) units are in kgco2e, so we convert to MT using 1e-9
production = (F_agg + F_y_agg).loc[impact] * 1e-9

# Consumption impacts (CBA): isolate the GHG impact column
consumption = (f.loc[impact] @ L @ Y_agg + F_y_agg.loc[impact]) * 1e-9

# Dimension check
print(f.loc[impact].shape, L.shape, Y_agg.shape, F_y_agg.loc[impact].shape)
print('q:', consumption.shape)

What are Canada's impacts?

In [ ]:
print(production['CA'], 'Mt')
print(consumption['CA'], 'Mt')

Interesting; note that Canada is a net exporter of CO2 https://www.canada.ca/en/environment-climate-change/services/environmental-indicators/carbon-dioxide-emissions-consumption-perspective.html
<br><br>
Finally, lets check that our calculationgs are correct: consumption and production footprints should be equal at a global level

In [ ]:
print(production.sum()/consumption.sum())

Plot emissions of global regions

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

impacts_df = pd.concat([consumption, production],axis=1)
impacts_df.columns = ['consumption GWP100 (Mt)','production GWP100 (Mt)']
impacts_df.plot.bar(ax=ax, width=0.8)

### This concludes the main section of the notebook.

--------------------------------------------------------
## 4. Extra exercise: Scope 1 2 3 accounting of industry
Once you have constructed the EEIO variables, it is easy to do a variety of analysis, like trade linkages, sectoral contributions to final demand, etc. Below is an example of how we can calculate scope 1 (primary production of emissions), 2 (+electricity), 3 (+consumption) emissions for regions in Exiobase. The idea is to become comfortable with navigating, aggregating, and subsetting the tables while applying the same basic formulas
<br><br>
note that for most IO models, you will want to save all of your tables to avoid having to recreate them every time you want to do new calculations. This does not take too long for exiobase (~3-5 minutes), but some models like OpenIO Canada can take ~40 minutes to construct all of the tables.
<br><br>
For this we will use the other impact tables: the "satellite" impact tables, which have the raw stressors (e.g. all different types of CO2 emissions) rather than the collected emissions. This is sometimes called the true $F$ in models whereas the impact tables we have been using so far are found from a $C \times F$ multiplication with an additional table $C$.

In [ ]:
Fr = pd.read_csv(f'{path}satellite/F.txt', sep='\t', index_col=[0], header=[0, 1])  
F_yr = pd.read_csv(f'{path}satellite/F_Y.txt', sep='\t', index_col=[0], header=[0, 1]) 

In [ ]:
# get the normalized values of f for above
fr = Fr @ inv_diag_x

In [ ]:
# isolate only the climate-related stressors
Fr_ghg = Fr.loc[fr.index.str.contains('CO2')].sum(0)
fr_ghg = fr.loc[fr.index.str.contains('CO2')].sum(0)

For this exercise, we also need to construct the Z table, and we need some indexes stored.

In [ ]:
# this may take a bit
Z = A @ np.diag(x)

In [ ]:
# Isolate some labels for what comes next (the region/sector index of the IO table)
A_lab = A.index.to_frame(index=None)
r_lab = A_lab.region.drop_duplicates().reset_index(drop=True)
s_lab = A_lab.sector.drop_duplicates()

# sizes
r = r_lab.shape[0]
s = s_lab.shape[0]

In the cell above we store indexes of the IO table as we will need to loop through them. In Exiobase pxp, products $128 \rightarrow 141$ are related to electrcity production, transmission, distribution.

In [ ]:
# create an index vector which encodes the position of the electricity sectors:
# (we can diagonalize later to get just these in a matmul)
elec = np.zeros(r * s)

# repeats every s, r times
for i in range(r):
    elec[i*s + 128-1: i*s + 141-1] = np.ones(141-128)

elec

`elec` is a vector that encodes the position of these electricity products for each country
<br><br> Below we calculate each scope. <b>Scope 1</b> is simply the direct emissions from territorial production, which is already encoded in $F$

In [ ]:
scope_1 = Fr_ghg * 1e-9 #.groupby('region').sum() # in kg -> Mt = * 10^-9
scope_1.head()

<b>Scope 2</b> involves adding electricity emissions to Scope 1. Here we multiply impacts by the Z table instead of LY because we are interested in the industry impacts, not final demand coming from that industry production

In [ ]:
# Scope 2 emissions: scope 1 emissions + the electricity emissions
# extracted from emissions embodied in electricity: fLzj -> extract via diag
scope_2 = (fr_ghg @ np.diag(elec) @ Z) * 10**-9
scope_2.index = pd.MultiIndex.from_frame(A_lab)
scope_2.head()

<b>Scope 3</b> still not interested in final demand, but we now take the impacts of all inter-industry activities for each contry. 

In [ ]:
# Scope 3 emissions: emissions from the rest of the supply chain (not just electricity)
# we find overall flz, not just electricity
scope_3 = (fr_ghg @ L @ Z) * 10**-9
scope_3.index = pd.MultiIndex.from_frame(A_lab)
scope_3.head()

In [ ]:
# plot them
scoped_emissions = pd.DataFrame(pd.concat([scope_1.groupby('region').sum(),
                                scope_2.groupby('region').sum(),
                                scope_3.groupby('region').sum()], axis=1),
                                )
scoped_emissions.columns = ['scope_1','scope_2','scope_3']


fig, ax = plt.subplots(figsize=(20,8))
scoped_emissions.sort_values('scope_1', ascending=False).plot.bar(ax=ax,width=0.85)